In [1]:
import sys
from pathlib import Path

# Add the path to the utils folder to sys.path
utils_path = Path('../python').resolve()
print(utils_path)
sys.path.append(str(utils_path))

from pathlib import Path
from rasterio.crs import CRS
from crop_utils import (crop_image_by_coordinates, 
                        crop_image_by_geojson_shp, 
                        reproject_geotiff)
from edit_path_utils import (edit_paths)

import rasterio
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from process_utils import (preprocess_data,
                           superpixel_segmentation,
                            tv_denoise, 
                            preprocess_for_merge)
from rio_utils import (reproject_arr_to_match_profile)

/shared/Wildfire/Bety_wildfire_project/Bety_A_S_code/Alpha_Code/wildfire_new/uavsar-main/python


In [ ]:
# data_dir = Path("/shared/Wildfire/Bety_wildfire_project/Bety_data_test/post_rectangle_45km_40km")
# tifs = sorted(list(data_dir.rglob('./*_alpha1.tif')))
# tifs

In [ ]:
# path_to_images = tifs
# center_longitude = -117.92976
# center_latitude = 34.33253
# width_km = 45
# height_km =40

# output_names = [str(data_dir) + '/' + file.stem + '_box_crop_' + str(width_km) +"km_"+ str(height_km) + 'km.tif' for file in path_to_images]
# print(*output_names, sep="\n")

In [ ]:
# for i in range(len(path_to_images)):
#     crop_image_by_coordinates(path_to_images[i],
#                               output_names[i], 
#                               center_longitude, 
#                               center_latitude, 
#                               width_km,
#                               height_km)

In [ ]:
# Opens a GeoTIFF and loads the backscatter values and profile
def open_one(path):
    with rasterio.open(path) as ds:
        band = ds.read(1)
        profile = ds.profile
    return band, profile

In [ ]:
def inc_filter(img):
    img_deg = np.rad2deg(img)
    mask = (img_deg < 25) | (img_deg > 65)
    img_deg[mask] = 999
    return img_deg

In [ ]:
def denoise(data, weight):
    mask = np.isnan(data)
    data[mask] = 9999
    data_tv = tv_denoise(data, weight)
    data_tv[mask] = np.nan

    return data_tv

In [ ]:
def weighted_inc_merge(img_0, img_1, inc_0, inc_1):
    
    valid_pixel_0 = ~np.isnan(img_0) & (inc_0 >= 25) & (inc_0 <= 65)
    valid_pixel_1 = ~np.isnan(img_1) & (inc_1 >= 25) & (inc_1 <= 65)
    invalid_pixel_0 = np.isnan(img_0) | (inc_0 < 25) | (inc_0 > 65)
    invalid_pixel_1 = np.isnan(img_1) | (inc_1 < 25) | (inc_1 > 65)

    w0 = inc_0/ (inc_0 + inc_1)
    w1 = inc_1 / (inc_0 + inc_1)
    img_merged = np.empty_like(img_0)
    img_merged[:] = np.nan

    # combine valid pixels
    img_merged[valid_pixel_0] = (w0 * img_0)[valid_pixel_0] + (w1 * img_1)[valid_pixel_0]
    img_merged[valid_pixel_1] = (w0 * img_0)[valid_pixel_1] + (w1 * img_1)[valid_pixel_1]

    # combine cases where one pixel is valid and the other is not
    img_merged[valid_pixel_0 & invalid_pixel_1] = img_0[valid_pixel_0 & invalid_pixel_1]
    img_merged[valid_pixel_1 & invalid_pixel_0] = img_1[valid_pixel_1 & invalid_pixel_0]

    return img_merged


In [ ]:
# #Dr.Uma Code 
# def weighted_inc_merge(img_0, img_1, inc_0, inc_1):

#     valid_pixel_0 = ~np.isnan(img_0) & (inc_0 >= 20) & (inc_0 <= 60)
#     valid_pixel_1 = ~np.isnan(img_1) & (inc_1 >= 20) & (inc_1 <= 60)
#     invalid_pixel_0 = np.isnan(img_0) | (inc_0 < 20) | (inc_0 > 60)
#     invalid_pixel_1 = np.isnan(img_1) | (inc_1 < 20) | (inc_1 > 60)

#     w0 = inc_0 / (inc_0 + inc_1)
#     w1 = inc_0 / (inc_1 + inc_1)
#     img_merged = np.empty_like(img_0)
#     img_merged[:] = np.nan

#     # combine valid pixels
#     # img_merged[valid_pixel_0] = (w0 * img_0)[valid_pixel_0] + (w1 * img_1)[valid_pixel_0]
#     # img_merged[valid_pixel_1] = (w0 * img_0)[valid_pixel_1] + (w1 * img_1)[valid_pixel_1]

#     # combine cases where one pixel is valid and the other is not
#     img_merged[valid_pixel_0 & invalid_pixel_1] = img_0[valid_pixel_0 & invalid_pixel_1]
#     img_merged[valid_pixel_1 & invalid_pixel_0] = img_1[valid_pixel_1 & invalid_pixel_0]
#     img_merged[invalid_pixel_0 & invalid_pixel_1] = (w0 * img_0)[invalid_pixel_0 & invalid_pixel_1] + (w1 * img_1)[invalid_pixel_0 & invalid_pixel_1]

#     return img_merged

In [ ]:
data_dir = Path("/shared/Wildfire/Bety_wildfire_project/Bety_data_test/post_rectangle_45km_40km")

tifs = sorted([tif for tif in data_dir.glob('*alpha1_box_crop_45km_40km.tif') if not tif.name.startswith('._')])
tifs

In [ ]:
tifs[0], tifs[2], tifs[1], tifs[3]

In [ ]:
data_dir = Path("/shared/Wildfire/Bety_wildfire_project/Bety_data_test/post_rectangle_45km_40km")

incs = sorted(list(data_dir.rglob('./*inc_box_crop_45km_40km*.tif')))
incs

In [ ]:
incs[0], incs[2], incs[1], incs[3]

In [ ]:
# def preprocess_for_merge(data: np.ndarray) -> np.ndarray:
#     data_masked = data.copy()
#     background_mask = (data_masked <= 0)
#     data_masked[background_mask] = np.nan
#     return data_masked


In [ ]:
bands, profiles = zip(*map(open_one, tifs))

bands = list(bands)
for i in range(len(bands)):
    bands[i] = preprocess_for_merge(bands[i]) # mask out the missing data after RTC. Scale the values to ensure consistency
    

post_south_w_to_e = bands[0]
post_north_w_to_e = bands[1]
post_south_e_to_w = bands[2]
post_north_e_to_w = bands[3]
profile_post_south_w_to_e = profiles[0]
profile_post_north_w_to_e = profiles[1]
profile_post_south_e_to_w = profiles[2]
profile_post_north_e_to_w = profiles[3]



In [ ]:
inc_bands, inc_profiles = zip(*map(open_one, incs))

inc_bands = list(inc_bands)
for i in range(len(inc_bands)):
    inc_bands[i] = inc_filter(inc_bands[i]) # convert angle from radian to degree. Filter out the bad angles

post_inc_south_w_to_e = inc_bands[0]
post_inc_north_w_to_e = inc_bands[1]
post_inc_south_e_to_w = inc_bands[2]
post_inc_north_e_to_w = inc_bands[3]
profile_inc_post_south_w_to_e = inc_profiles[0]
profile_inc_post_north_w_to_e = inc_profiles[1]
profile_inc_post_south_e_to_w = inc_profiles[2]
profile_inc_post_north_e_to_w = inc_profiles[3]

In [ ]:
resampling = 'bilinear'

In [ ]:
# Reprojecting all the images to match the profile of pre_0. 
# We would need to perform addition and division later so the shape has to match

post_south_e_to_w , _ = reproject_arr_to_match_profile(post_south_e_to_w , profile_post_south_e_to_w , profile_post_south_w_to_e, resampling=resampling)
post_south_e_to_w  = post_south_e_to_w[0]
post_north_w_to_e, _ = reproject_arr_to_match_profile(post_north_w_to_e, profile_post_north_w_to_e, profile_post_south_w_to_e, resampling=resampling)
post_north_w_to_e = post_north_w_to_e[0]
post_north_e_to_w, _ = reproject_arr_to_match_profile(post_north_e_to_w, profile_post_north_e_to_w, profile_post_south_w_to_e, resampling=resampling)
post_north_e_to_w = post_north_e_to_w[0]

post_inc_south_w_to_e, _ = reproject_arr_to_match_profile(post_inc_south_w_to_e, profile_inc_post_south_w_to_e, profile_post_south_w_to_e , resampling=resampling)
post_inc_south_w_to_e = post_inc_south_w_to_e[0]
post_inc_south_e_to_w, _ = reproject_arr_to_match_profile(post_inc_south_e_to_w, profile_inc_post_south_e_to_w, profile_post_south_w_to_e, resampling=resampling)
post_inc_south_e_to_w = post_inc_south_e_to_w[0]

post_inc_north_w_to_e, _ = reproject_arr_to_match_profile(post_inc_north_w_to_e, profile_inc_post_south_e_to_w, profile_post_south_w_to_e , resampling=resampling)
post_inc_north_w_to_e = post_inc_north_w_to_e[0]
post_inc_north_e_to_w, _ = reproject_arr_to_match_profile(post_inc_north_e_to_w, profile_inc_post_north_e_to_w, profile_post_south_w_to_e , resampling=resampling)
post_inc_north_e_to_w = post_inc_north_e_to_w[0]

In [ ]:
# note we will get error message here as we are dividing everything, 
# but those pixels with invaid value encountered were not used for the merge.
# we only used the valid pixels for merging which should not cause error in division since 20 < inc < 60


alpha_south = weighted_inc_merge(post_south_w_to_e, post_south_e_to_w, post_inc_south_w_to_e, post_inc_south_e_to_w)
alpha_north = weighted_inc_merge(post_north_w_to_e, post_north_e_to_w , post_inc_north_w_to_e, post_inc_north_e_to_w)

In [ ]:
output_path_0 = "/shared/Wildfire/Bety_wildfire_project/Bety_data_test/post_rectangle_45km_40km/bobcat_alpha_0.tif"
output_path_1 = "/shared/Wildfire/Bety_wildfire_project/Bety_data_test/post_rectangle_45km_40km/bobcat_alpha_1.tif"

In [ ]:
with rasterio.open(output_path_0, "w", **profile_post_south_w_to_e) as dest:
    dest.write(alpha_south, 1)

with rasterio.open(output_path_1, "w", **profile_post_south_w_to_e) as dest:
    dest.write(alpha_north, 1)


In [ ]:
plt.imshow((alpha_south))

In [ ]:
plt.imshow((alpha_north))  

In [ ]:
print(alpha_north.shape)

In [ ]:
print(alpha_south.shape)

In [ ]:
post_alpha_merged = np.nanmean(np.stack([alpha_south, alpha_north]), axis=0)


In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Merged Post Alpha Image")
plt.imshow(np.log10(post_alpha_merged))
#plt.colorbar(label='log10(backscatter)')
plt.show()


In [ ]:


# # ---- Save merged alpha to GeoTIFF ----
# save_path = "/shared/Wildfire/Bety_wildfire_project/Bety_data_test/post_rectangle_45km_40km/post_merged_alpha_image_test.tif"

# # Use one of the inputs to copy geospatial metadata
# with rasterio.open(output_path_0) as src:
#     profile = src.profile

# # Update profile for the merged image
# profile.update({
#     "height": post_alpha_merged.shape[0],
#     "width": post_alpha_merged.shape[1],
#     "count": 1,
#     "dtype": post_alpha_merged.dtype,
#     "compress": "lzw"
# })

# # Save merged Alpha raster
# with rasterio.open(save_path, "w", **profile) as dst:
#     dst.write(post_alpha_merged, 1)



In [ ]:
# import rasterio
# import numpy as np

# def print_tif_values(tif_path):
#     """
#     Print the numerical values and unique values in a TIF file.

#     Parameters:
#     tif_path (str): Path to the TIF file
#     """
#     try:
#         # Open the TIF file
#         with rasterio.open(tif_path) as src:
#             # Print basic metadata
#             print(f"TIF File: {tif_path}")
#             print(f"Width: {src.width}, Height: {src.height}")
#             print(f"Number of bands: {src.count}")
#             print(f"Data type: {src.dtypes[0]}")

#             # Read all bands
#             for band_idx in range(1, src.count + 1):
#                 band_data = src.read(band_idx)

#                 print(f"\nBand {band_idx} data:")

#                 # If the array is too large, print a sample
#                 if band_data.size > 100:
#                     print("Array is large, showing statistics and sample:")
#                     print(f"Min: {np.nanmin(band_data)}, Max: {np.nanmax(band_data)}, Mean: {np.nanmean(band_data):.4f}, Median: {np.nanmedian(band_data):.4f}")
#                     print("Sample of data (top-left corner):")
#                     print(band_data[:5, :5])  # Show a 5x5 sample from the top left
#                 else:
#                     # If the array is small enough, print all values
#                     print(band_data)

#                 # Extract valid (non-NaN) values and round
#                 valid = band_data[~np.isnan(band_data)]
#                 if valid.size > 0:
#                     unique_vals = np.unique(np.round(valid, 2))
#                     print("Unique values (non-NaN, rounded to 2 decimals):")
#                     print(unique_vals[:50])  # Show first 50 for brevity
#                 else:
#                     print("No valid (non-NaN) values found in this band.")

#     except Exception as e:
#         print(f"Error reading TIF file: {e}")


# # Call your function
# tif_file_path = "/shared/Wildfire/Bety_wildfire_project/Bety_data_test/Post_alpha_perimeter_data/post_merged_alpha_image_test.tif"
# print_tif_values(tif_file_path)


In [ ]:
# # === Save Postfire alpha1 ===
# post_alpha1_out_path = "/shared/Wildfire/Bety_wildfire_project/Bety_data_test/Post_alpha_perimeter_data/post_merged_alpha_image_test_aligned.npy"
# np.save(post_alpha1_out_path, post_alpha_merged)